# Testes com a GCN implementada no Kaggle

Foi encontrada uma implementação da GCN utilizada no artigo ["Anti-Money Laundering in Bitcoin: Experimenting with Graph Convolutional Networks for Financial Forensics"][1], porém implementada em Pytorch puro. Essa implementação foi [encontrada no Kaggle][2]. Verficou-se que não há diferença significativa nos resultados das versões do Pytorch Geometric e essa versão.

[1]: https://arxiv.org/pdf/1908.02591.pdf
[2]: https://www.kaggle.com/karthikapv/gcn-elliptic-dataset 

In [1]:
import torch

In [2]:
train_data = []
test_data = []

for i in range(1,35):
    train_data.append(torch.load('elliptic_pt/train/' + str(i) + '.pt'))

for i in range(35,50):
    test_data.append(torch.load('elliptic_pt/test/' + str(i) + '.pt'))

In [3]:
train_data

[Data(x=[2147, 166], edge_index=[2, 1924], y=[2147], adjacency_matrix=[2147, 2147]),
 Data(x=[1117, 166], edge_index=[2, 858], y=[1117], adjacency_matrix=[1117, 1117]),
 Data(x=[1279, 166], edge_index=[2, 727], y=[1279], adjacency_matrix=[1279, 1279]),
 Data(x=[1440, 166], edge_index=[2, 1169], y=[1440], adjacency_matrix=[1440, 1440]),
 Data(x=[1882, 166], edge_index=[2, 1491], y=[1882], adjacency_matrix=[1882, 1882]),
 Data(x=[485, 166], edge_index=[2, 209], y=[485], adjacency_matrix=[485, 485]),
 Data(x=[1203, 166], edge_index=[2, 858], y=[1203], adjacency_matrix=[1203, 1203]),
 Data(x=[1165, 166], edge_index=[2, 1044], y=[1165], adjacency_matrix=[1165, 1165]),
 Data(x=[778, 166], edge_index=[2, 484], y=[778], adjacency_matrix=[778, 778]),
 Data(x=[972, 166], edge_index=[2, 538], y=[972], adjacency_matrix=[972, 972]),
 Data(x=[696, 166], edge_index=[2, 477], y=[696], adjacency_matrix=[696, 696]),
 Data(x=[506, 166], edge_index=[2, 446], y=[506], adjacency_matrix=[506, 506]),
 Data(x=

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import sys

class GraphConv(nn.Module):
    def __init__(self, in_features, out_features, activation  = 'relu', skip = False, skip_in_features = None):
        super(GraphConv, self).__init__()
        self.W = torch.nn.Parameter(torch.DoubleTensor(in_features, out_features))
        nn.init.xavier_uniform_(self.W)
        
        self.set_act = False
        if activation == 'relu':
            self.activation = nn.ReLU()
            self.set_act = True
        elif activation == 'softmax':
            self.activation = nn.Softmax(dim = 1)
            self.set_act = True
        else:
            self.set_act = False
            raise ValueError("activations supported are 'relu' and 'softmax'")
            
        self.skip = skip
        if self.skip:
            if skip_in_features == None:
                raise ValueError("pass input feature size of the skip connection")
            self.W_skip = torch.nn.Parameter(torch.DoubleTensor(skip_in_features, out_features)) 
            nn.init.xavier_uniform_(self.W)
        
    def forward(self, A, H_in, H_skip_in = None):
        # A must be an n x n matrix as it is an adjacency matrix
        # H is the input of the node embeddings, shape will n x in_features
        self.A = A
        self.H_in = H_in
        A_ = torch.add(self.A, torch.eye(self.A.shape[0]).double())
        D_ = torch.diag(A_.sum(1))
        # since D_ is a diagonal matrix, 
        # its root will be the roots of the diagonal elements on the principle diagonal
        # since A is an adjacency matrix, we are only dealing with positive values 
        # all roots will be real
        D_root_inv = torch.inverse(torch.sqrt(D_))
        A_norm = torch.mm(torch.mm(D_root_inv, A_), D_root_inv)
        # shape of A_norm will be n x n
        
        H_out = torch.mm(torch.mm(A_norm, H_in), self.W)
        # shape of H_out will be n x out_features
        
        if self.skip:
            H_skip_out = torch.mm(H_skip_in, self.W_skip)
            H_out = torch.add(H_out, H_skip_out)
        
        if self.set_act:
            H_out = self.activation(H_out)
            
        return H_out

In [5]:
class GCN_2layer(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, skip = False):
        super(GCN_2layer, self).__init__()
        self.skip = skip
        
        self.gcl1 = GraphConv(in_features, hidden_features)
        
        if self.skip:
            self.gcl_skip = GraphConv(hidden_features, out_features, activation = 'softmax', skip = self.skip,
                                  skip_in_features = in_features)
        else:
            self.gcl2 = GraphConv(hidden_features, out_features, activation = 'softmax')
        
    def forward(self, A, X):
        out = self.gcl1(A, X)
        if self.skip:
            out = self.gcl_skip(A, out, X)
        else:
            out = self.gcl2(A, out)
            
        return out

In [6]:
num_features = 166
num_classes = 2
epochs = 1000
lr = 0.001


# 0 - illicit, 1 - licit
#labels_ts = []
#for c in classes_ts:
#    labels_ts.append(np.array(c['class'] == '2', dtype = np.long))

gcn = GCN_2layer(num_features, 100, num_classes)
train_loss = nn.CrossEntropyLoss(weight = torch.DoubleTensor([0.7, 0.3]))
optimizer = torch.optim.Adam(gcn.parameters(), lr = lr)

# Training


    
for ep in range(epochs):
    for ts, data in enumerate(train_data):
        A = data.adjacency_matrix
        X = data.x
        L = data.y
        t_start = time.time()
        
        gcn.train()
        optimizer.zero_grad()
        out = gcn(A, X)

        loss = train_loss(out, L)
        train_pred = out.max(1)[1].type_as(L)
        acc = (train_pred.eq(L).double().sum())/L.shape[0]

        loss.backward()
        optimizer.step()

        sys.stdout.write("\r Epoch %d/%d Timestamp %d training loss: %f training accuracy: %f Time: %s"
                         %(ep, epochs, ts+1, loss, acc, time.time() - t_start)
                        )

 Epoch 999/1000 Timestamp 34 training loss: 0.319192 training accuracy: 0.996117 Time: 0.082777023315429698

In [7]:
import os

torch.save(gcn.state_dict(), str(os.path.join("./modelDir", "gcn_weights.pth")))

In [8]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report


gcn = GCN_2layer(num_features, 100, num_classes)
gcn.load_state_dict(torch.load(os.path.join("./modelDir", "gcn_weights.pth")))

# Testing
test_accs = []
test_precisions = []
test_recalls = []
test_f1s = []
total_L = []
total_test_pred = []


for data in test_data:
    A = data.adjacency_matrix
    X = data.x
    L = data.y
    
    gcn.eval()
    test_out = gcn(A, X)
    
    test_pred = test_out.max(1)[1].type_as(L)
    t_acc = (test_pred.eq(L).double().sum())/L.shape[0]
    test_accs.append(t_acc.item())
    test_precisions.append(precision_score(L, test_pred))
    test_recalls.append(recall_score(L, test_pred))
    test_f1s.append(f1_score(L, test_pred))
    total_L += L
    total_test_pred += test_pred

print(classification_report(total_L, total_test_pred))

acc = np.array(test_accs).mean()
prec = np.array(test_precisions).mean()
rec = np.array(test_recalls).mean()
f1 = np.array(test_f1s).mean()

print("GCN - averaged accuracy: {}, precision: {}, recall: {}, f1: {}".format(acc, prec, rec, f1))

              precision    recall  f1-score   support

           0       0.74      0.50      0.60      1083
           1       0.97      0.99      0.98     15587

    accuracy                           0.96     16670
   macro avg       0.86      0.74      0.79     16670
weighted avg       0.95      0.96      0.95     16670

GCN - averaged accuracy: 0.9492094623532064, precision: 0.9589358137173803, recall: 0.9875010187796687, f1: 0.9727337866538667


In [9]:
test_data

[Data(x=[1341, 166], edge_index=[2, 1002], y=[1341], adjacency_matrix=[1341, 1341]),
 Data(x=[1708, 166], edge_index=[2, 1148], y=[1708], adjacency_matrix=[1708, 1708]),
 Data(x=[498, 166], edge_index=[2, 423], y=[498], adjacency_matrix=[498, 498]),
 Data(x=[756, 166], edge_index=[2, 653], y=[756], adjacency_matrix=[756, 756]),
 Data(x=[1183, 166], edge_index=[2, 1055], y=[1183], adjacency_matrix=[1183, 1183]),
 Data(x=[1211, 166], edge_index=[2, 1180], y=[1211], adjacency_matrix=[1211, 1211]),
 Data(x=[1132, 166], edge_index=[2, 1048], y=[1132], adjacency_matrix=[1132, 1132]),
 Data(x=[2154, 166], edge_index=[2, 1443], y=[2154], adjacency_matrix=[2154, 2154]),
 Data(x=[1370, 166], edge_index=[2, 935], y=[1370], adjacency_matrix=[1370, 1370]),
 Data(x=[1591, 166], edge_index=[2, 1497], y=[1591], adjacency_matrix=[1591, 1591]),
 Data(x=[1221, 166], edge_index=[2, 1346], y=[1221], adjacency_matrix=[1221, 1221]),
 Data(x=[712, 166], edge_index=[2, 388], y=[712], adjacency_matrix=[712, 712